# Лабораторно-конкурсная работа 2. Композиции алгоритмов. Ранжирование.

Данная работа состоит из двух этапов: лабораторного и конкурсного. Для обоих используется один и тот же набор данных, который вы можете найти в описании первого конкурсного задания на kaggle.

Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете так же должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.

Наиболее предпочтительный подход к решению «лабоконкурса» - проделать предлагаемые лабораторные задания, а затем приступить к участию в конкурсном этапе. Это позволит вам перед участием в конкурсе познакомиться с представленными данными, освоить полезные приемы для улучшения качества, получить бейзлайн решение и глубже вникнуть в суть темы. 

Однако, мы не запрещаем вам выполнить только один из двух этапов: вы можете проделать лабораторные исследования и не участвовать в конкурсе или же пропустить лабораторную часть и сразу заняться соревновательной.

Мы не предоставляем код бейзлайн решения конкурса. Идеи для бейзлайна вы можете почерпнуть из заданий лабораторной части.

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 15 баллов. Сдавать задание после указанного срока сдачи нельзя. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, нам необходима ссылка на источник).

Преодоление medium baseline на kaggle дает вам 5 баллов за конкурс. Если вы выполните задания из лабораторной, то пробить medium baseline для вас не составит труда.

### Правила сдачи
Выполненную работу следует отправить в систему Anytask. Более подробно о системе можно почитать на странице курса. Название отправляемого файла должно иметь следующий формат: Surname_Name_Group_NN.ipynb, где NN — номер лабораторной работы. Например, Kozlova_Anna_CS_02.ipynb.

**(2 балла) Задание 1.** Загрузите данные о запросах и их релевантности (*train.csv*, *test.csv*), а также информацию об организациях (*train_org_information.json*, *test_org_information.json*) и рубриках (*train_rubric_information.json*, *test_rubric_information.json*)

Для дальнейших экспериментов необходимо посчитать небольшой набор базовых факторов. С использованием информации о запросах и организациях, посчитайте факторы, которые на ваш взгляд будут полезными для предсказания релевантности.

Примерами текстовых факторов могут служить:
 - кол-во слов в запросе и названии организации;
 - пословные/N-граммные пересечения слов запроса и названия организации (также можно использовать синонимы названия организации и адрес организации): кол-во слов в пересечении, [мера Жаккара](https://en.wikipedia.org/wiki/Jaccard_index) и пр.;
 - кол-во различных синонимичных названий организации (поле *names* в описании организации);
 - One-hot-encoded язык запроса.
 
По информации о географическом положении:
 - факт совпадения региона, где задавался запрос и региона организации;
 - координаты показанной области;
 - размеры показанной области;
 - меры, характеризующие близость координат организации к показанному окну: расстояние до центра области и другие.
 
Факторы, описывающие организацию:
 - one-hot-encoding фактор cтраны или региона организации (важно: не используйте one-hot-encoding факторы, в которых больше 10 значений; если в факторе слишком много значений, ограничьтесь, например, только самыми популярными категориями)
 - кол-во рабочих дней в неделе и общая продолжительность работы (поле *work_intervals* в описании организации)
 - кол-во рубрик (поле *rubrics* в описании организации)
 
![](https://miro.medium.com/max/1500/0*FwubnnoNlt6Coo9j.png)

В этом задании не нужно использовать многомерные представления текстовой информации (tfidf и прочие embeddings) и информацию о кликах (*train_clicks_information.json*, *test_clicks_information.json*). Придумывать сверхсложные факторы тоже необязательно.

Вы можете реализовать описанные выше факторы и/или придумать свои. Для экспериментов в лабораторной - достаточно реализовать предложенное, для победы в конкурсе - вряд ли. Но зачастую такие простые признаки могут приносить наибольшую пользу.

В итоге у вас должно получиться от 15 до 50 факторов, характеризующих запрос и организацию и покрывающих основные источники данных (кроме кликов). Это наш основной датасет, который будет использоваться в экспериментах.

**Важно**: До раздела *«Ранжирование»* будем считать, **что решается задача *регрессии* (предсказываем абсолютное значение релевантности)**.

In [1]:
import json
import langdetect
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import Counter
from sklearn.preprocessing import OneHotEncoder

In [2]:
train_data = pd.read_csv('competition-2-shad-fall-2018/train.csv')
train_data.head(5)

,query_id,query,region,org_name,org_id,window_center,window_size,relevance
0,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Суд Жовтневого району міста Дніпропетровськ,1021049127,"34.613119,48.506531","0.025928,0.017380",0.0
1,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Дніпропетровський окружний адміністративний суд,1602348889,"34.613119,48.506531","0.025928,0.017380",0.0
2,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Бабушкінський районний суд,1105837793,"34.613119,48.506531","0.025928,0.017380",0.0
3,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Красногвардійський районний суд,1066267658,"34.613119,48.506531","0.025928,0.017380",0.0
4,11,"суд, Украина, Днепропетровская область, Днепро...",21775,Жовтневий суд,1661586235,"34.613119,48.506531","0.025928,0.017380",0.0


In [3]:
with open('competition-2-shad-fall-2018/train_org_information.json', 'r') as read_file:
    train_org_json = json.load(read_file)

In [4]:
def getWordsFromString(x) -> str:
    return re.sub(r"[^A-Za-zА-Яа-я]+", ' ', x).lower().split()

In [5]:
def getIntersectionWordsCount(x) -> int:
    return len(set(getWordsFromString(x[0])) & set(getWordsFromString(x[1])))

In [6]:
def getSynonymousNamesCount(x) -> int:
    node = train_org_json.get(str(x))
    if node is not None:
        return len(node["names"])
    return 0

In [7]:
def getEncodedLangs():
    langs = train_data['query'].apply(lambda x: langdetect.detect(x))
    sortedLangs = [lang for lang, _ in Counter(langs).most_common()]
    langsEncoder = OneHotEncoder(sparse=False).fit(langs.values.reshape(-1,1))
    encodedLangs = langsEncoder.transform(langs.values.reshape(-1,1))
    langsIdx = [np.where(langsEncoder.categories_[0] == lang)[0][0] for lang in sortedLangs[:10]]
    
    newEncodedLangs = np.zeros((encodedLangs.shape[0], 11))
    for i, idx in enumerate(langsIdx):
        newEncodedLangs[:, i] = encodedLangs[:, idx]
    for row in newEncodedLangs:
        if row.sum() == 0:
            row[-1] = 1
    return newEncodedLangs

In [8]:
def getRubricsCount(x) -> int:
    node = train_org_json.get(str(x))
    if node is not None:
        return len(node["rubrics"])
    return 0

In [9]:
def getWorkTime(x) -> int:
    node = train_org_json.get(str(x))
    if node is not None:
        work_interval_sum = 0
        for work_interval in node["work_intervals"]:
            work_interval_sum += work_interval["time_minutes_end"] - work_interval["time_minutes_begin"]
        if len(node["work_intervals"]) != 0:
            return work_interval_sum / len(node["work_intervals"])
    return 0

In [10]:
def getDaysCount(x):
    node = train_org_json.get(str(x))
    if node is not None:
        daysCount = 0
        for work_interval in node["work_intervals"]:
            day = work_interval["day"]
            if day == "everyday":
                daysCount = max(daysCount, 7)
            elif day == "weekdays":
                daysCount = max(daysCount, 5)
            elif day == "weekend":
                daysCount = max(daysCount, 2)
            elif day in ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]:
                daysCount = 1
            else:
                daysCount = 0
        return daysCount
    return 0

In [11]:
def getRegionCode(x) -> str:
    node = train_org_json.get(str(x))
    if node is not None:
        return node["address"]["region_code"]
    return 0

In [12]:
def getEncodedRegionCode():
    regionCodes = train_data["org_id"].apply(getRegionCode).values
    sortedRegionCodes = [code for code, _ in Counter(regionCodes).most_common()[:10]]
    codesEncoder = OneHotEncoder(sparse=False).fit(regionCodes.reshape(-1,1))
    encodedCodes = codesEncoder.transform(regionCodes.reshape(-1,1))
    codesIdx = [np.where(codesEncoder.categories_[0] == code)[0][0] for code in sortedRegionCodes[:10]]
    
    newEncodedCodes = np.zeros((encodedCodes.shape[0], 11))
    for i, idx in enumerate(codesIdx):
        newEncodedCodes[:, i] = encodedCodes[:, idx]
    for row in newEncodedCodes:
        if row.sum() == 0:
            row[-1] = 1
    return newEncodedCodes

In [ ]:
new_train_data = pd.DataFrame()

new_train_data['query_words_count'] = train_data['query'].apply(lambda query: len(getWordsFromString(query)))
new_train_data['org_name_words_count'] = train_data['org_name'].apply(lambda name: len(getWordsFromString(name)))
new_train_data['intersection_words_count'] = train_data[['query', 'org_name']].apply(getIntersectionWordsCount, axis=1)
new_train_data['jaccard_score'] = new_train_data['intersection_words_count'] / (new_train_data['query_words_count'] + new_train_data['org_name_words_count'] - new_train_data['intersection_words_count'])
new_train_data['synonymous_names_count'] = train_data['org_id'].apply(getSynonymousNamesCount)

encodedLangs = getEncodedLangs()
new_train_data = pd.concat([new_train_data, pd.DataFrame(encodedLangs)], axis=1)

In [ ]:
new_train_data['window_center1'] = train_data['window_center'].apply(lambda x: x.split(',')[0]).values
new_train_data['window_center2'] = train_data['window_center'].apply(lambda x: x.split(',')[1]).values
new_train_data['window_size1'] = train_data['window_size'].apply(lambda x: x.split(',')[0]).values
new_train_data['window_size2'] = train_data['window_size'].apply(lambda x: x.split(',')[1]).values
new_train_data['rubrics_count'] = train_data['org_id'].apply(getRubricsCount)
new_train_data['worktime'] = train_data['org_id'].apply(getWorkTime)
new_train_data['workdays_count'] = train_data['org_id'].apply(getDaysCount)

In [ ]:
encodedCodes = getEncodedRegionCode()
new_train_data = pd.concat([new_train_data, pd.DataFrame(encodedCodes, columns=list(range(11,22)))], axis=1)

In [ ]:
new_train_data["relevance"] = train_data["relevance"]

In [ ]:
new_train_data

## Bias-Variance decomposition. Композиции алгоритмов

![](http://scott.fortmann-roe.com/docs/docs/BiasVariance/biasvariance.png)

Рассмотрим задачу регрессии со среднеквадратичной функцией потерь, а также некоторый алгоритм $a$. Тогда качество алгоритма $a$ может быть записано следующим образом:

$$Q(a) = \mathbb{E}_{X^l} \mathbb{E}_{x,y}(a(x) - y)^2,$$

где первое матожидание вычисляется по всевозможным обучающим выборкам $X^l$ размера $l$. К сожалению, на реальных данных эта формула неприменима из-за невозможности сгенерировать необходимые для оценки данные. Поэтому проведем приближенный численный эксперимент с эмпирическими оценками матожиданий.

С помощью бутстраппинга можно просемплировать из обучающей выборки $N$ новых выборок того же размера, тем самым "имитируя" пространство всевозможных обучающих выборок, после чего обучить на каждой выбранный алгоритм. Обозначим вектор истинных меток тестовой выборки за $y \in \mathbb{R}^{m}$. Векторы прогнозов для объектов из тестовой выборки для каждой модели обозначим за $\hat{y}_i \in \mathbb{R}^{m}, i \in \{1, .., N\}$. Тогда средний квадрат ошибки по всем моделям на тестовой выборке запишется как

$$error=\frac{1}{N}\sum_{i=1}^{N}MSE(y,\hat{y}_i).$$

Обозначим среднее предсказание за $$\overline{y} = \frac{1}{N}\sum_{i=1}^{N} \hat{y}_i.$$

Тогда квадрат отклонения среднего предсказания и разброс прогнозов относительно среднего предсказания всех моделей на тестовой выборке от истинных меток запишутся следующим образом, соответственно:

$$bias^2 = MSE(y, \overline y),$$

$$variance = \frac{1}{N}\sum_{i=1}^N MSE(\hat{y}_i, \overline y).$$


Для начала рассмотрим в качестве алгоритма решающее дерево. Как известно, при увеличении высоты дерева алгоритм может быть сильно чувствителен к составу обучающей выборки. Чтобы подтвердить эти предположения, проведите следующие эксперименты.


Разбейте обучающий пул с факторами на обучающую и тестовую выборки в пропорциях 1 к 4 соответственно. Обратите внимание, что обучающая выборка меньше тестовой. Такая большая тестовая выборка позволит сделать измерение качества моделей достаточно достоверным. 

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_splited, test_splited = train_test_split(new_train_data.values, train_size=1/5)
train_splited.shape[0] / test_splited.shape[0]

In [ ]:
train_splited_size = train_splited.shape[0]
train_splited_size

In [ ]:
test_splited_size = test_splited.shape[0]
test_splited_size

**(1 балл) Задание 2.** Постройте графики зависимости $error$, $bias^2$ и $variance$ от глубины решающего дерева (от 1 до 15 включительно) для $N=200$. 

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor

In [ ]:
depths = list(range(1,16))

In [ ]:
N = 200
boots_trains_idx = np.random.choice(list(range(train_splited_size)), (train_splited_size, N), replace=True)
boots_trains_idx.shape

In [ ]:
boots_trains = []
for train_idx in np.transpose(boots_trains_idx):
    boots_trains.append(train_splited[train_idx])
boots_trains = np.array(boots_trains)

In [ ]:
boots_trains.shape

In [ ]:
errors = []
bias2s = []
variances = []

X_test = test_splited[:, :-1]
y_test = test_splited[:, -1]

for depth in depths:
    mean_y_pred = np.zeros(test_splited_size)
    mse_sum = 0
    y_preds = []
    for train in boots_trains:
        X_train = train[:, :-1]
        y_train = train[:, -1]
        decision_tree_regressor = DecisionTreeRegressor(max_depth=depth).fit(X_train, y_train)
        y_pred = np.array(decision_tree_regressor.predict(X_test))
        y_preds.append(y_pred)
        mse_sum += mean_squared_error(y_test, y_pred)
        mean_y_pred += y_pred
    mean_y_pred /= N
    errors.append(mse_sum / N)
    bias2s.append(mean_squared_error(y_test, mean_y_pred))
    
    var = 0
    for y in y_preds:
        var += mean_squared_error(y, mean_y_pred)
    variances.append(var / N)

In [ ]:
plt.plot(depths, errors, label="$Error$")
plt.plot(depths, bias2s, label="$Bias^2$")
plt.plot(depths, variances, label="$Variance$")
plt.xlabel('depth')
plt.ylabel('error')
plt.legend()
plt.show()

**(0.5 балла) Задание 3.** Являются ли какие-то из полученных графиков монотонными? А должны ли они быть монотонными, если бы гипотетически эксперименты были проведены на всевозможных выборках? Почему? Убедитесь численно, что верно bias-variance разложение ошибки: $$error = bias^2 + variance$$

In [ ]:
np.array(errors) - (np.array(bias2s) + np.array(variances))

**Вывод:**  
- $bias^2$ и $variance$ являются монотонными.  
- разность между ошибкой и суммой смещения и разброса отличаются на $10^{-18}$, поэтому разложение верно

## Композиции алгоритмов

Несмотря на описанный выше недостаток решающих деревьев, объединение их в композиции позволяет существенно улучшить качество предсказания. Рассмотрим несколько способов построения композиций.

### Bagging + RSM

![](https://sites.google.com/site/rajhansgondane2506/_/rsrc/1467898300734/publications/rrftrain.jpg?height=215&width=320)

Один из способов объединения алгоритмов в композиции — обучение каждого отдельного алгоритма на некоторой подвыборке из исходной выборки ([bagging](https://en.wikipedia.org/wiki/Bootstrap_aggregating)) и подмножестве исходных признаков ([RSM](https://en.wikipedia.org/wiki/Random_subspace_method)). В sklearn этот тип композиции реализован в классе [BaggingRegressor](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingRegressor.html) (для случая регресии). Подобный подход также есть в реализации [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html).

### Градиентный бустинг

В случае бустинга композиция алгоритмов строится последовательно. Каждый следующий базовый алгоритм акцентируется на тех объектах, на которых обученная ранее композиция допускала ошибку.

Туториал по работе с известными реализациями градиентного бустинга XGBoost и CatBoost можно найти на вики-странице курса.

**(1 балл) Задание 4.** Проведите аналогичный эксперимент с bias-variance разложением для *градиентного бустинга*, а также для *случайного леса*, варьируя количество базовых алгоритмов (рассмотрите 1, 5, 10, 25, 50, 100 решающих деревьев).

Используйте модели градиентного бустанга XGBoost или CatBoost. Random Forest возьмите из пакета sklearn.

In [ ]:
estimators = [1, 5, 10, 25, 50, 100]

#### XGBoost

In [ ]:
import xgboost

In [ ]:
errorsXGB = []
bias2sXGB = []
variancesXGB = []

X_test = test_splited[:, :-1]
y_test = test_splited[:, -1]

for estimator in estimators:
    mean_y_pred = np.zeros(test_splited_size)
    mse_sum = 0
    y_preds = []
    for train in boots_trains:
        X_train = train[:, :-1]
        y_train = train[:, -1]
        xgbRegressor = xgboost.XGBRegressor(n_estimators=estimator).fit(X_train, y_train)
        y_pred = np.array(xgbRegressor.predict(X_test))
        y_preds.append(y_pred)
        mse_sum += mean_squared_error(y_test, y_pred)
        mean_y_pred += y_pred
    mean_y_pred /= N
    errorsXGB.append(mse_sum / N)
    bias2sXGB.append(mean_squared_error(y_test, mean_y_pred))
    
    var = 0
    for y in y_preds:
        var += mean_squared_error(y, mean_y_pred)
    variancesXGB.append(var / N)

In [ ]:
plt.plot(estimators, errorsXGB, label="$Error$")
plt.plot(estimators, bias2sXGB, label="$Bias^2$")
plt.plot(estimators, variancesXGB, label="$Variance$")
plt.xlabel('estimators')
plt.ylabel('error')
plt.legend()
plt.show()

#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
errorsRF = []
bias2sRF = []
variancesRF = []

X_test = test_splited[:, :-1]
y_test = test_splited[:, -1]

for estimator in estimators:
    mean_y_pred = np.zeros(test_splited_size)
    mse_sum = 0
    y_preds = []
    for train in boots_trains:
        X_train = train[:, :-1]
        y_train = train[:, -1]
        random_forest_regressor = RandomForestRegressor(n_estimators=estimator).fit(X_train, y_train)
        y_pred = np.array(random_forest_regressor.predict(X_test))
        y_preds.append(y_pred)
        mse_sum += mean_squared_error(y_test, y_pred)
        mean_y_pred += y_pred
    mean_y_pred /= N
    errorsRF.append(mse_sum / N)
    bias2sRF.append(mean_squared_error(y_test, mean_y_pred))
    
    var = 0
    for y in y_preds:
        var += mean_squared_error(y, mean_y_pred)
    variancesRF.append(var / N)

In [ ]:
plt.plot(estimators, errorsRF, label="$Error$")
plt.plot(estimators, bias2sRF, label="$Bias^2$")
plt.plot(estimators, variancesRF, label="$Variance$")
plt.xlabel('estimators')
plt.ylabel('error')
plt.legend()
plt.show()

**(0.5 балла) Задание 5.** Отличаются ли графики в рассмотренных моделях (градиентный бустинг, случайный лес)  между собой? На какую компоненту из разложения ошибки влияет объединение алгоритмов в рассмотренный тип композиции? Поясните свой ответ.

**Вывод:** Рассмотренные модели строят последовательно алгоритмы, каждый из которых будет уменьшать ошибку предыдущего, то есть он последовательно старается уменьшить смещение и дисперсию. На графиках можно увидеть, что на ошибку больше всего влияет смещение. Однако, XGBoost справляется с данной задачей намного лучше (с точки зрения уменьшения смещения) по сравнению со случайным лесом. В случайном лесе же лучше уменьшается дисперсия.

### Стекинг
![](https://4.bp.blogspot.com/-hCxAb57kzDQ/VuMgHy3hAhI/AAAAAAAAAVk/djmL9IHv5QkLWeudjE50qDoCTbiUrTetA/s1600/Stacking.jpg)

[Stacking](https://en.wikipedia.org/wiki/Ensemble_learning#Stacking) — еще один способ объединить несколько алгоритмов в один, который часто используется как в решении реальных задач из промышленной сферы, так и в конкурсах на платформах вроде Kaggle. Подход использует понятие *базовых классификаторов*, каждый из которых независимо обучается на некотором (возможно одном и том же) множестве признаков, а также *мета-классификатора*, использующего предсказания базовых классификаторов как факторы. 


**(1 балл) Задание 6.** Использование мета-классификатора подразумевает получение предсказаний от базовых классификаторов для тех объектов обучающей выборки, на которых мета-классификатор будет обучаться. В свою очередь базовые классификаторы тоже должны быть обучены на некоторой выборке. Чтобы избежать переобучения, обучающее множество делится на $n$ фолдов, $(n-1)$ из которых используются для обучения базовых классификаторов, а $n$-ый — для предсказания (вычисления значений мета-фактора).

Для получения мета-факторов для тестовых данных базовые классификаторы могут быть обучены на всем обучающем множестве, поскольку проблема переобучения здесь не возникает. Другими словами, если мы хотим посчитать факторы для тестового множества, мы можем спокойно использовать обучающее множество для тренировки базовых классификаторов. Если же мы хотим посчитать факторы для обучающего множества, то необходимо следить, чтобы классификатор не предсказывал для тех объектов, на которых обучался.

Мета-классификатор может быть обучен как на множестве исходных факторов, дополненным мета-факторами, так и исключительно на множестве мета-факторов. Выбор зависит от решаемой задачи.

Напишите функцию, которая получает на вход классификатор, обучающую и тестовые выборки, а также параметры [кросс-валидатора](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) и возвращающую значения мета-фактора для обучающего и тестового множеств.

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
def compute_meta_feature(clf, X_train, X_test, y_train, cv, args, kwargs):
    testMetaFeature = clf.fit(X_train, y_train).predict_proba(X_test)[:, 1]
    
    trainMetaFeature = np.zeros(y_train.shape)
    kfold = KFold(*args, **kwargs)
    for trainIdx, testIdx in kfold.split(X_train):
        X = X_train[trainIdx]
        y = y_train[trainIdx]
        trainMetaFeature[testIdx] = clf.fit(X, y).predict_proba(X_train[testIdx])[:, 1]
        
    return testMetaFeature.reshape(-1,1), trainMetaFeature.reshape(-1,1)

**(1 балл) Задание 7.** Обучите различные известные вам модели машинного обучения и сделайте из них стекинг-композицию. 
  
  Базовые алгоритмы могут отличаться друг от друга:
    - моделью машинного обучения,
    - гиперпараметрами (например, различные функции потерь или глубины деревьев),
    - набором факторов,
    - целевой переменной: исходного целевую переменную y можно заменить на функцию от него (sqr(y), sqrt(y), log(y + 1), ...), бинаризовать по порогу ([y>y0]) или произвести другое преобразование.
  
  Обратите внимание, что бинарные мета-факторы дают меньше полезного сигнала мета-классификатору, чем числовые, поэтому базовым классификаторам лучше возвращать вероятность/числовую функцию, чем метки классов.


In [ ]:
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
X_clf, y_clf = make_classification(n_samples=10000, n_features=10, n_informative=4, random_state=541)
train_clf = np.hstack((X_clf, y_clf.reshape(-1,1)))
train_clf, test_clf = train_test_split(train_clf, train_size=0.8)

In [ ]:
X_train = train_clf[:, :-1]
y_train = train_clf[:, -1]

X_test = test_clf[:, :-1]
y_test = test_clf[:, -1]

In [ ]:
def getMetaFeatures(clfs, X_train, X_test, y_train, args, kwargs):
    metaFeature = compute_meta_feature(clfs[0], X_train, X_test, y_train, _, args, kwargs)
    testMeta = metaFeature[0]
    trainMeta = metaFeature[1]
    for clf in clfs[1:]:
        metaFeature = compute_meta_feature(clf, X_train, X_test, y_train, _, args, kwargs)
        testMeta = np.hstack((testMeta, metaFeature[0]))
        trainMeta = np.hstack((trainMeta, metaFeature[1]))
    return testMeta, trainMeta

In [ ]:
def stackingOnTestMetaFeature(clfs, metaClf, X_train, X_test, y_train, y_test, args, kwargs): 
    X, _ = getMetaFeatures(clfs, X_train, X_test, y_train, args, kwargs)
    metaClf.fit(X, y_test)

In [ ]:
def stackingOnTrainMetaFeature(clfs, metaClf, X_train, X_test, y_train, y_test, args, kwargs): 
    _, X = getMetaFeatures(clfs, X_train, X_test, y_train, args, kwargs)
    metaClf.fit(X, y_train)

#### Пример 1

In [ ]:
clfs = [LogisticRegression(solver="sag"), RandomForestClassifier(n_estimators=70)]
metaClf = KNeighborsClassifier()

In [ ]:
stackingOnTestMetaFeature(clfs, metaClf, X_train, X_test, y_train, y_test.reshape(-1,1), (6,), {"random_state": 789})

In [ ]:
X = getMetaFeatures(clfs, X_train ,X_test, y_train, (6,), {"random_state": 500})

In [ ]:
"score on test: {}".format(metaClf.score(X[0], y_test.reshape(-1,1)))

In [ ]:
"score on train: {}".format(metaClf.score(X[1], y_train.reshape(-1,1)))

#### Пример 2

In [ ]:
clfs = [DecisionTreeClassifier(max_depth=4), DecisionTreeClassifier(max_depth=20), DecisionTreeClassifier(max_depth=30)]
metaClf = RandomForestClassifier()

In [ ]:
stackingOnTestMetaFeature(clfs, metaClf, X_train, X_test, y_train, y_test.reshape(-1,1), (6,), {"random_state": 523})

In [ ]:
X = getMetaFeatures(clfs, X_train ,X_test, y_train, (6,), {"random_state": 423})

In [ ]:
"score on test: {}".format(metaClf.score(X[0], y_test.reshape(-1,1)))

In [ ]:
"score on train: {}".format(metaClf.score(X[1], y_train.reshape(-1,1)))

#### Пример 3

In [ ]:
clfs = [DecisionTreeClassifier(max_depth=4), DecisionTreeClassifier(max_depth=20), DecisionTreeClassifier(max_depth=30)]
metaClf = RandomForestClassifier()

In [ ]:
stackingOnTrainMetaFeature(clfs, metaClf, X_train, X_test, y_train, y_test.reshape(-1,1), (6,), {"random_state": 523})

In [ ]:
X = getMetaFeatures(clfs, X_train ,X_test, y_train, (6,), {"random_state": 423})

In [ ]:
"score on test: {}".format(metaClf.score(X[0], y_test.reshape(-1,1)))

In [ ]:
"score on train: {}".format(metaClf.score(X[1], y_train.reshape(-1,1)))

**(1 балл) Задание 8.** В этом задании мы попробуем обучить стекинг композицию градиентного бустинга и линейной модели на данных различной природы.

Загрузите данные по кликам (*train_clicks_information.json, test_clicks_information.json*). Используя текстовые информацию из данных о кликах, организациях, рубриках и запросах, постройте bag-of-words представление с помощью TfidfVectorizer.

После этого у вас будет два набора факторов: базовые факторы (*base*), реализованные ранее, и разреженная матрица признаков, полученная применением TfidfVectorizer'а (*tfidf*).

Обучите различные модели и замерьте качество на тестовой выборке по метрике MSE:

 1. Линейная модель на tfidf факторах;
 
 2. Линейная модель на tfidf + base факторах;

 3. Градиентный бустинг на base факторах;

 4. Градиентный бустинг на tfidf + base факторах (здесь можно сократить размерность tfidf, если данные не умещаются в памяти)
 
 5. Градиентный бустинг на base факторах + мета-факторе «предсказание модели из п.1»
 

Можете воспользоваться любой из двух реализаций градиентного бустинга: XGBoost или CatBoost.

Для тренировки tfidf модели можно использовать и обучающие, и тестовые данные, поскольку она не зависит от целевой переменной.


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
with open('competition-2-shad-fall-2018/train_clicks_information.json', 'r') as read_file:
    train_clicks_json = json.load(read_file)

In [15]:
with open('competition-2-shad-fall-2018/train_rubric_information.json', 'r') as read_file:
    train_rubric_json = json.load(read_file)

In [ ]:
train_data.head(5)

In [16]:
def getClicks(x)->str:
    node = train_clicks_json.get(str(x))
    if node is not None:
        return " ".join(node)
    return ""

In [17]:
def getOrgData(x)->str:
    node = train_org_json.get(str(x))
    if node is not None:
        names = ""
        for d in node["names"]:
            names += d["value"]["value"] + " "
        
        rubrics = ""
        for rubric in node["rubrics"]:
            rubrics += getRubricData(rubric) + " "
        
        address = node["address"]["formatted"]["value"]
        return names + rubrics + address
    return ""

In [18]:
def getRubricData(x):
    node = train_rubric_json.get(str(x))
    if node is not None:
        keywords = ""
        for value in node["keywords"]:
            keywords += value["value"] + " "
        
        phrases = ""
        for phrase in node["phrases"]:
            phrases += phrase["value"] + " "
        
        descriptions = ""
        for d in node["descriptions"]:
            descriptions += d["value"]["value"] + " "
        
        names = ""
        for d in node["names"]:
            names += d["value"] + " "
        
        return keywords + phrases + descriptions + names
    return ""

In [19]:
def getWords(row) -> str:
    query = row[0]
    org_name = row[1]
    clicks_data = getClicks(row[2])
    org_data = getOrgData(row[2])
    return query + " " + org_name + " " + clicks_data + " " + org_data

In [20]:
corpus = train_data[['query', 'org_name', 'org_id']].apply(getWords, axis=1)

In [28]:
corpus.values

array(["суд, Украина, Днепропетровская область, Днепродзержинский городской совет Суд Жовтневого району міста Дніпропетровськ суд кіровського району дніпропетровська суды г днепропетровска заводской суд г днепродзержинска жовтневій суд днепропетровск украина днепропетровск суд суд на паторжинского днепропетровск жовтневий суд м дніпропетровська улица поторжинского суд жовтневого района г днепропетровска официальный сайт суд г днепр жовтневый районный суд жовтневый районный суд днепр Sud Zhovtnevogo rayona goroda Dnepropetrovsk Суд Жовтневого района города Днепропетровск Суд Жовтневого району Суд Жовтневого району міста Дніпропетровськ Суд Октябрского района hukuki, adliye, adliyeler, işlemler, islemler, sarayı, adalet, mahkeme, sarayları, mahkemeler Суды Amtsgericht Courthouses Gericht Gerichtsbehörde Gerichtshof Justice Courts Mahkemeler Palais de Justice Rechtsprechung Richter Tribunal adalet sarayı adliye adliye sarayları adliyeler autorité judiciaire autorités judiciaires cour de j

In [ ]:
all_words = c

In [24]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(corpus.values).toarray()

In [29]:
tfidf.shape

(29274, 203321)

In [30]:
from sklearn.decomposition import PCA

In [ ]:
tfidf = PCA(n_components=1000).fit_transform(tfidf)

In [ ]:
relevance = train_data['relevance'].values.reshape(-1, 1)

In [ ]:
tfidf.shape

У меня модель не обучается на полном tfidf, поэтому оставлю только 2000 рандомных признаков

In [ ]:
Idx = np.random.choice(tfidf.shape[1], size=2000, replace=False)
Idx

#### Линейная модель на tfidf факторах

In [ ]:
from sklearn.linear_model import ElasticNet, SGDRegressor
from sklearn.svm import SVR

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf[:, Idx], relevance[:, Idx], train_size=1/5)

In [ ]:
sgd = SGDRegressor().fit(X_train, y_train)

#### Линейная модель на tfidf + base факторах

#### Градиентный бустинг на base факторах

#### Градиентный бустинг на tfidf + base факторах (здесь можно сократить размерность tfidf, если данные не умещаются в памяти)

#### Градиентный бустинг на base факторах + мета-факторе «предсказание модели из п.1»

**(1 балл) Задание 9.**
Проанализируйте результаты, полученные в предыдущем задании. Сравните модели с точки зрения полученного качества, времени обучения и затрат по памяти.

Удалось ли улучшить качество предсказания с помощью стекинга?

Почему реализованная в п.5 стекинг композиция может работать лучше, чем градиентный бустинг, обученный на всем множестве факторов (п.4)?

Помогло ли добавление базовых факторов в линейную модель (модель из п.2 против модели из п.1)? Почему?

## Ранжирование

![](http://i.imgur.com/2QnD2nF.jpg)

Задачу поискового ранжирования можно описать следующим образом: имеется множество документов $d \in D$ и множество запросов $q \in Q$. Требуется оценить *степень релевантности* документа по отношению к запросу: $(q, d) \mapsto r$, относительно которой будет производиться ранжирование. Для восстановления этой зависимости используются методы машинного обучения. Обычно используется три типа:
 - признаки запроса $q$, например: мешок слов текста запроса, его длина, ...
 - документа $d$, например: значение PageRank, мешок слов, доменное имя, ...
 - пары $(q, d)$, например: число вхождений фразы из запроса $q$ в документе $d$, ...

Одна из отличительных особенностей задачи ранжирования от классических задач машинного обучения заключается в том, что качество результата зависит не от предсказанных оценок релевантности, а от порядка следования документов в рамках конкретного запроса, т.е. важно не абсолютное значение релевантности (его достаточно трудно формализовать в виде числа), а то, более или менее релевантен документ, относительно других документов.
### Подходы к решению задачи ранжирования
Существуют 3 основных подхода, различие между которыми в используемой функции потерь:
  
1. **Pointwise подход**. В этом случае рассматривается *один объект* (в случае поискового ранжирования - конкретный документ) и функция потерь считается только по нему. Любой стандартный классификатор или регрессор может решать pointwise задачу ранжирования, обучившись предсказывать значение таргета. Итоговое ранжирование получается после сортировки документов к одному запросу по предсказанию такой модели.
2. **Pairwise подход**. В рамках данной модели функция потерь вычисляется по *паре объектов*. Другими словами, функция потерь штрафует модель, если отражированная этой моделью пара документов оказалась в неправильном порядке.
3. **Listwise подход**. Этот подход использует все объекты для вычисления функции потерь, стараясь явно оптимизировать правильный порядок.

### Оценка качества

Для оценивания качества ранжирования найденных документов в поиске используются асессорские оценки. Само оценивание происходит на скрытых от обучения запросах $Queries$. Для этого традиционно используется метрика *DCG* ([Discounted Cumulative Gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain)) и ее нормализованный вариант — *nDCG*, всегда принимающий значения от 0 до 1.
Для одного запроса DCG считается следующим образом:
$$ DCG = \sum_{i=1}^P\frac{(2^{rel_i} - 1)}{\log_2(i+1)}, $$

где $P$ — число документов в поисковой выдаче, $rel_i$ — релевантность (асессорская оценка) документа, находящегося на i-той позиции.

*IDCG* — идеальное (наибольшее из возможных) значение *DCG*, может быть получено путем ранжирования документов по убыванию асессорских оценок.

Итоговая формула для расчета *nDCG*:

$$nDCG = \frac{DCG}{IDCG} \in [0, 1].$$

Чтобы оценить значение *nDCG* на выборке $Queries$ ($nDCG_{Queries}$) размера $N$, необходимо усреднить значение *nDCG* по всем запросам  выборки:
$$nDCG_{Queries} = \frac{1}{N}\sum_{q \in Queries}nDCG(q).$$

Пример реализации метрик ранжирование на python можно найти [здесь](https://gist.github.com/mblondel/7337391).

В рамках нашей задачи «документом» будет являться организация.

Разбейте обучающую выборку на обучение и контроль в соотношении 70 / 30. Обратите внимание, что разбивать необходимо множество запросов, а не строчки датасета.

Далее рассмотрим несколько подходов предсказания релевантности. Для оценивания качества моделей используйте метрику nDCG на контроле. В случае подбора гиперпараметров используйте кросс-валидацию по 5 блокам, где разбиение должно быть по запросам, а не строчкам датасета.

###  Ранжируем с XGBoost и CatBoost

XGBoost имеет несколько функций потерь для решения задачи ранжирования:
1. **reg:linear** — данную функцию потерь можно использовать для решения задачи ранжирование *pointwise* подходом.
2. **rank:pairwise** — в качестве *pairwise* модели в XGBoost реализован [RankNet](http://icml.cc/2015/wp-content/uploads/2015/06/icml_ranking.pdf), в котором минимизируется гладкий функционал качества ранжирования: $$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = log(1 + e^{-M}), $$ где $ a(x) $ - функция ранжирования. Суммирование ведется по всем парам объектов, для которых определено отношение порядка, например, для пар документов, показанных по одному запросу. Таким образом функция потерь штрафует за то, что пара объектов неправильно упорядочена.
3. **rank:map, rank:ndcg** — реализация [LambdaRank](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf) для двух метрик: [MAP](https://en.wikipedia.org/wiki/Information_retrieval#Mean_average_precision) и **nDCG**. Известно, что для того, чтобы оптимизировать негладкий функционал, такой как **nDCG**,  нужно домножить градиент функционала $ Obj(a) $ на значение $\Delta NDCG_{ij} $ — изменение значения функционала качества при замене $x_i$ на $ x_j$.  Поскольку для вычисления метрик необходимы все объекты выборки, то эти две ранжирующие функции потерь являются представителями класса *listwise* моделей.

Реализованные в CatBoost ранжирующие функции потерь можной найти [здесь](https://catboost.ai/docs/concepts/loss-functions-ranking.html#groupwise-metrics).

**(2 балла) Задание 10.** Попробуйте различные функции потерь (регрессионные и ранжирующие) для моделей XGBoost и CatBoost. Настройте основные параметры моделей (глубина, кол-во деревьев, глубина, скорость обучения, регуляризация).  
Сравните построенные модели с точки зрения метрики nDCG на контроле и проанализируйте полученные результаты:
  - какая модель работает лучше всего для данной задачи? 
  - в чем достоинства/недостатки каждой? 
  - сравните модели между собой: 
   - получается ли сравнимое качество линейного pointwise подхода с остальными моделями? 
   - заметна ли разница в качестве при использовании бустинга с разными функциями потерь?

**(1 балл) Задание 11.** Одним из основных преимуществ CatBoost'a является обработка категориальных факторов «из коробки». Добавьте в датасет различные категориальные факторы из данных и обучите заново CatBoost модели. Улучшилось ли качество?

#### Пользовательская функция потерь

Библиотека XGBoost позволяет использовать пользовательские функции потерь. Для этого необходимо реализовать функцию, принимающую на вход вектор предсказанных значений и обучающую выборку, и возвращающую градиент и гессиан, посчитанный по входным данным.

Важно отметить, что XGBoost использует диагональную аппроксимацию гессиана, таким образом все недиагональные элементы считаются малозначимыми и приравниваются нулю, поэтому и градиент, и гессиан являются векторами длины размера обучающей выборки.

**(3 балла) Задание 12.** Реализуйте экспоненциальную функцию потерь для XGBoost:
$$ Obj = \sum_{i \prec j} \mathcal{L}\left(a(x_j) - a(x_i)\right) \rightarrow min $$ $$ \mathcal{L}(M) = e^{-M} $$

Обучите модель с помощью данной функции потерь, настройте параметры.

**Комментарии к реализации**

В случае ранжирования XGBoost'у необходимо знать о разбиении всех объектов на группы. В нашем случае в одну группу будут входить документы, соответствующие одному запросу. Функция, считающая градиент и гессиан по данным, должна знать данное разбиение датасета. Однако питоновский интерфейс класса *DMatrix* (в котором хранится датасет) не дает возможности получить это разбиение. В этом случае нужно реализовать функцию потерь в качестве функтора, конструктор которого принимает разбиение на группы в качестве параметра.

Пример реализации своей функции потерь можно найти на соответствующем семинаре.

In [ ]:
class ExponentialPairwiseLoss(object):
    def __init__(self, groups):
        self.groups = groups
                        
    def __call__(self, preds, dtrain):
        # your code here
        pass

**Задание 13.** С помощью наилучшей из получившихся в ходе лабораторной работы моделей сделайте предсказания для тестовой части выборки и отправьте их на kaggle. В отдельной ячейке выпишите получившийся результат (NDCG).

**Результат:**

-----------------------------------------------------------

На этом лабораторная часть задания закончена. Далее - конкурсная часть. Вы можете решать его прямо здесь или в отдельном ноутбуке. Эта часть работы проверяется только у решений, попавших в топ в лидерборде.
Если решать конкурс в отдельном ноутбуке удобнее, можете выложить его в энитаск рядом с решением лабораторной.

Удачи!

Кратко опишите, как вы добились такого результата. Расскажите, что успели попробовать и что из этого сработало.

Кроме этого, можете оставить отзыв о лабораторно-конкурсном задании :)